## Tokenización de texto para calcular costos de uso de OpenAI

### Instalación de titktoken

In [2]:
!pip install tiktoken

In [3]:
import tiktoken

### Selección y carga de encoding

cl100k_base para modelos gpt-4, gpt-3.5-turbo, text-embedding-ada-002.

Consulta el <a>https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb</a>  para otro tipo de modelos o cambios en este encoding.

In [4]:
encoding = tiktoken.get_encoding("cl100k_base")

In [5]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

### Tokenización de string

In [6]:
encoding.encode("El perrito come solito.")

[6719, 281, 618, 6491, 2586, 2092, 6491, 13]

In [7]:
len(encoding.encode("El perrito come solito."))

8

### Tokenizar una serie de mensajes

In [8]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens  

In [9]:
example_messages = [
    {
      "role": "system",
      "content": "Eres un asistente de atención a clientes y estudiantes de la plataforma de educación online en tecnología, inglés y liderazgo llamada Platzi"
    },
    {
      "role": "user",
      "content": "¿Cómo puedo mejorar mis habilidades de creación de contenido con inteligencia artificial?"
    },
    {
      "role": "assistant",
      "content": "Para mejorar tus habilidades de creación de contenido con inteligencia artificial te sugiero que tomes el Taller de Creación de Contenido con Inteligencia Artificial en https://platzi.com/cursos/contenido-ia/. En este curso aprenderás a utilizar herramientas para generar contenido de manera automatizada y eficiente. ¡No te lo pierdas!"
    }
]

In [10]:
num_tokens_from_messages(example_messages, model="gpt-3.5-turbo-0613")

139

## Preparación de datos y análisis para fine-tuning de chat model

<a>https://cookbook.openai.com/examples/chat_finetuning_data_prep</a>

In [11]:
import json
import tiktoken # para conteo de Tokens
import numpy as np
from collections import defaultdict

### Cargar dataset desde archivo JSONL

In [16]:
data_path = "./data_train.jsonl"

In [17]:
# Cargar dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Análisis inicial del dataset
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 1261
First example:
{'role': 'system', 'content': 'Eres un asistente de atención a clientes y estudiantes de la plataforma de educación online en tecnología, inglés y liderazgo llamada Platzi'}
{'role': 'user', 'content': '¿Cómo puedo mejorar mis habilidades en JavaScript?,'}
{'role': 'assistant', 'content': 'Para mejorar tus habilidades en JavaScript, te recomendamos tomar el Curso Práctico de JavaScript en https://platzi.com/cursos/javascript-practico/. También puedes practicar en proyectos personales y buscar recursos en línea para continuar aprendiendo. ¡Mucho éxito en tu camino de aprendizaje de JavaScript!'}


### Validacion del Formato

In [18]:
# Revisión de errores de formato
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


### Utilidades de conteo de Tokens

In [19]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

### Conteo de Tokens y advertencias de data

In [20]:
# Advertencias y conteo de tokens
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 68, 206
mean / median: 133.87946074544013, 133.0
p5 / p95: 114.0, 159.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 9, 132
mean / median: 69.51704996034893, 69.0
p5 / p95: 50.0, 92.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


### Estimación de costos

In [21]:
# Costo y estimado de número de épocas por defecto
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~168822 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~506466 tokens


## Importar librería de OpenAI

In [23]:
from openai import OpenAI
client = OpenAI(api_key='INGRESA TU API KEY')

### Cargar datasets
Carga los archivos de los datasets a la plataforma de OpenAI. Esto te generará un ID del archivo que usarás para el fine-tuning del modelo base.

### Dataset de entrenamiento

In [ ]:
client.files.create(
  file=open("INGRESA EL PATH DE data_train.jsonl", "rb"),
  purpose="fine-tune"
)

### Dataset de validacion

In [ ]:
client.files.create(
  file=open("INGRESA EL PATH DE data_val.jsonl", "rb"),
  purpose="fine-tune"
)

### Fine-tuning de modelo base
Crea un proceso de fine-tuning.

Necesitas el id de tus dos archivos de datasets de entrenamiento y validación.
Además del nombre del modelo base.
Este código iniciará el proceso de fine-tuning que podrás darle seguimiento en la plataforma de OpenAI.

In [ ]:
client.fine_tuning.jobs.create(
  training_file="INGRESA EL FILE ID DEL ARCHIVO DE TRAIN", 
  validation_file='INGRESA EL FILE ID DEL ARCHIVO DE VAL',
  model="gpt-3.5-turbo-1106" #Puedes cambiar el modelo base según lo necesites.
)

### Listar y eliminar modelos con fine-tuning

### Listar modelos en tu organizacion

In [ ]:
client.fine_tuning.jobs.list(limit=10)

### Eliminar modelos de tu organizacion

In [ ]:
client.models.delete("INGRESA EL NOMBRE DEL MODELO A ELIMINAR")

## Análisis del modelo con fine-tunning

Fuente: <a>https://platform.openai.com/docs/guides/fine-tuning/analyzing-your-fine-tuned-model</a>

In [27]:
!pip install openai

### Uso de modelo con fine-tuning

In [ ]:
## Código exportado del Playground de OpenAI

from openai import OpenAI
client = OpenAI(api_key='INSERTA TU API KEY')

In [ ]:
print(response.choices[0].message.content)

### Carga de modelo con fine-tuning

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='INSERTA TU API KEY')

client.fine_tuning.jobs.retrieve("INSERTA EL JOB ID DE TU MODELO CON FINE-TUNING")

### Obtener archivo de resultados de fine-tuning de modelo

In [ ]:
content = client.files.content('INSERTA EL NOMBRE DE TU ARCHIVO DE METRICAS')

content.text

### Interpretación de resultados

In [ ]:
import pandas as pd

In [ ]:
metrics_str = content.text

In [ ]:
metrics_list = [line.split(',') for line in metrics_str.split('\n')]

In [ ]:
df = pd.DataFrame(metrics_list[1:], columns=metrics_list[0])
df = df.apply(pd.to_numeric, errors='coerce')
df.tail()

In [ ]:
#conversión a numéricos para graficación
df = df.apply(pd.to_numeric, errors='coerce')
df.tail()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,4))
plt.plot(df['step'], df['train_accuracy'])
plt.title('Training Accuracy over Steps')
plt.xlabel('Step')
plt.ylabel('Training Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(df['step'], df['train_loss'])
plt.title('Training Loss over Steps')
plt.xlabel('Step')
plt.ylabel('Training Accuracy')
plt.show()